In [ ]:
# import off-the-shelf libraries
import os
import imp
import sys
import GPy, scipy
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import time as timemodule
import copy

print '%s\t\t %s' % ('pandas',pd.__version__)
print '%s\t\t %s' % ('numpy', np.__version__)
print '%s\t\t %s' % ('scipy', scipy.__version__)
print '%s\t\t %s' % ('GPy', GPy.__version__)
print '%s\t\t %s' % ('seaborn', sns.__version__)
print '%s\t %s' % ('matplotlib', mpl.__version__)

# set global parameters
%matplotlib inline
sns.set_style('white')

# import in-house library
sys.path.append("..")
from libs import classical,growth,plates

***WARNING*** ```BiopythonExperimentalWarning```
<br></br></br> 
```Bio.phenotype``` is an experimental submodule which may undergo significant changes prior to its future official release.

# Read Data

In [ ]:
data = pd.read_csv('/Users/firasmidani/rab_fm/proj/biolog/data/Magellan/PRB954_PM1-1.tsv',
                   sep='\t',header=0,index_col=0);
data.head()

In [ ]:
# format 
print type(data.columns[0])

# convert headers from strings to integers
data.columns = plates.listTimePoints(interval=600,numTimePoints=data.shape[1])

data.index.name = 'Well'
data.T.index.name = 'Time'

print type(data.columns[0])

data.head()

# Read Meta-data

In [4]:
# libpath = '/Users/firasmidani/rab_fm/git/phenotypic-characterization/config'

# #libpath = os.path.dirname(os.path.realpath(__file__)) # get script's directory
# foo = imp.load_source('biolog_pm_layout','%s/biolog_pm_layout.py' % libpath);
# from biolog_pm_layout import *

# Carbon1

In [6]:
key = plates.populatePlateKey('PRB954_PM1-1');
key.head()

,Isolate,Substrate,Plate
Well,,,
A1,PRB954,Negative Control,PRB954_PM1-1
A2,PRB954,L-Arabanose,PRB954_PM1-1
A3,PRB954,N-Acetyl-D-Glucosamine,PRB954_PM1-1
A4,PRB954,D-Saccharic Acid,PRB954_PM1-1
A5,PRB954,Succinic Acid,PRB954_PM1-1


In [ ]:
# remove time points with NaN from data
data = data.iloc[:,np.where(~data.isna().all(0))[0]]
data.head()

In [ ]:
# initialize pd.DataFrame for summarizing data sets

order_columns = ['Letter','Plate','Row','Column']
order_columns += ['Isolate','Substrate','Max OD','Growth Fold']
order_columns

summary = summarizeGrowthData(data)
summary = summary.join(key)
summary = summary.loc[:,order_columns]

summary.head()

In [ ]:
# prepare data for growth dynamics inference
data = data.T
data = data.loc[:,key.index]
data = data.reset_index(drop=False);
data.head()

# Create GrowthPlate object and describe

In [ ]:
# define plate object and prepare it for analysis
plate = GrowthPlate(data=data,key=summary,control='A1');

In [ ]:
plate.time.head()

In [ ]:
plate.data.head()

In [ ]:
fig,ax = plate.plot();
fig

# Create GrowthData object and describe

In [ ]:
plate.mods

In [ ]:
fig,ax = plate.extractGrowthData({'Substrate':['alpha-D-Glucose']}).plot()

ax.set_ylim([-0.03,1.63])
ax.set_xlim([-1500,62000])

[ii.set(rotation=90) for ii in ax.get_xticklabels()];

ax.set_xlabel('Time (seconds)');

In [ ]:
plate.mods

## Conver Time to minutes

In [ ]:
plate.convertTimeUnits()

fig,ax = plate.extractGrowthData({'Substrate':['alpha-D-Glucose']}).plot()

ax.set_ylim([-0.03,1.63])
ax.set_xlim([-0.4,20.4])

ax.set_xlabel('Time (hours)');

In [ ]:
plate.mods

## Smooth OD data

In [ ]:
plate.smoothData();

fig,ax = plate.extractGrowthData({'Substrate':['alpha-D-Glucose']}).plot();

ax.set_ylim([-0.03,1.63])
ax.set_xlim([-0.4,20.4])

ax.set_xlabel('Time (hours)');

In [ ]:
plate.mods

## Transform with natural logarithm

In [ ]:
plate.logData();

fig,ax = plate.extractGrowthData({'Substrate':['alpha-D-Glucose']}).plot();

ax.set_ylim([-1.43,0.23])
ax.set_xlim([-0.4,20.4])

ax.set_xlabel('Time (hours)');

In [ ]:
plate.mods

## Scale OD to the first time point log(OD)=0 <--> OD=1

In [ ]:
plate.subtractBaseline();

fig,ax = plate.extractGrowthData({'Substrate':['alpha-D-Glucose']}).plot();

ax.set_ylim([-0.03,1.63])
ax.set_xlim([-0.4,20.4])

In [ ]:
plate.mods

## Subtract negative control OD at each time point

In [ ]:
plate.subtractControl();

fig,ax = plate.extractGrowthData({'Substrate':['alpha-D-Glucose']}).plot();

ax.set_ylim([-0.03,1.63])
ax.set_xlim([-0.4,20.4])

In [ ]:
plate.mods

In [ ]:
foo0 = imp.load_source('growth_models',lib_path_0);
foo1 = imp.load_source('plate_reader_library',lib_path_1);
foo2 = imp.load_source('growth_fitting_library',lib_path_2)

from growth_models import *
from plate_reader_library import *
from growth_fitting_library import *

## Fit growth curves with classical approach of Gompertz

In [ ]:
glucose = plate.extractGrowthData({'Substrate':['alpha-D-Glucose']})

classical = GrowthMetrics(glucose);
classical.Classical(logistic)
classical.inferClassicalDynamics()
classical.predictClassical()

fig,ax = classical.plot();

ax.set_ylim([-0.03,1.32])
ax.set_xlim([-0.4,20.4])

print 'Doubling Rate is %0.2f minutes' % classical.key['classical_td']

classical.key

## Fit growth curve with non-parameteric GP Regression approach

In [ ]:
glucose = plate.extractGrowthData({'Substrate':['alpha-D-Glucose']})

gpr = GrowthMetrics(glucose);
gpr.GP()
gpr.inferGPDynamics()
gpr.predictGP()

fig,ax = gpr.plot();

ax.set_ylim([-0.03,1.32])
ax.set_xlim([-0.4,20.4])

print 'Doubling Rate is %0.2f minutes' % gpr.key['GP_td']

gpr.key

# Summarize dynamics in all wells of the plate

In [ ]:
# initialize pd.DataFrame to store summary metrics/statistics
growth_summary = pd.DataFrame(index=summary.index,
                              columns=['GP_r','GP_K','GP_AUC','GP_td',
                                       'classical_r','classical_K','classical_AUC','classical_td']);
growth_summary.head()

In [ ]:
counter = 0;
row_count = 1; print '%02d' % row_count,
for well in growth_summary.index:

    if counter<12:
        print '.',
        counter += 1;
    else:
        row_count += 1;
        print '\n%02d .' % row_count,
        counter = 1;

    substrate = plate.key.loc[well,'Substrate']; #print substrate
    growth = plate.extractGrowthData({'Substrate':[substrate]}); # control is not subtracted
    
    
    classical = GrowthMetrics(growth)
    classical.Classical(gompertz)
    classical.inferClassicalDynamics()

    to_header = ['classical_r','classical_K','classical_AUC','classical_td'];
    to_index = classical.key.index[0]
    growth_summary.loc[to_index,to_header] = classical.key.loc[to_index,to_header].values

    gpr = GrowthMetrics(growth)
    gpr.GP()
    gpr.inferGPDynamics()

    to_header = ['GP_r','GP_K','GP_AUC','GP_td'];
    to_index = gpr.key.index[0]
    growth_summary.loc[to_index,to_header] = gpr.key.loc[to_index,to_header].values


In [ ]:
summary_df = summary.join(growth_summary).sort_values(['GP_td'],ascending=True)
summary_df.head(10)

# Analyze growth metrics across plate  (example)

In [ ]:
fig,ax = plt.subplots(figsize=[5,7]);

# grab all growth curves with at least 1.5 fold change (relative to negative control)
subset = list(summary_df[summary_df['Growth Fold']>1.5].Substrate.values)+list(['Negative Control']);

labels = summary_df[summary_df.isin({'Substrate':subset}).any(1)].Substrate
td = summary_df[summary_df.isin({'Substrate':subset}).any(1)].GP_td;

ax.barh(y=range(len(labels)),width=td,height=0.8,color=(0,0,0,0.65));

[ii.set(fontsize=20) for ii in ax.get_xticklabels()+ax.get_yticklabels()];

ax.set_xlabel('Doubling Time (Estimated by GP)',fontsize=20);
plt.setp(ax,yticks=range(len(labels)),yticklabels=labels);
ax.yaxis.grid(False)

In [ ]:
plate.mods

In [ ]:
fig,ax = plate.plot();
fig

## Would this be different if you do not subtract negative control (no carbon) growth?

In [ ]:
# define plate object and prepare it for analysis
plate = GrowthPlate(data=data,key=summary,control='A1');

In [ ]:
plate.convertTimeUnits()
plate.smoothData()
plate.logData()
plate.subtractBaseline()

plate.mods

In [ ]:
fig,ax = plate.plot(); fig

In [ ]:
# initialize pd.DataFrame to store summary metrics/statistics
growth_summary = pd.DataFrame(index=summary.index,
                              columns=['GP_r','GP_K','GP_AUC','GP_td',
                                       'classical_r','classical_K','classical_AUC','classical_td']);
growth_summary.head()

In [ ]:
counter = 0;
row_count = 1; print '%02d' % row_count,
for well in growth_summary.index:

    if counter<12:
        print '.',
        counter += 1;
    else:
        row_count += 1;
        print '\n%02d .' % row_count,
        counter = 1;

    substrate = plate.key.loc[well,'Substrate']; #print substrate
    growth = plate.extractGrowthData({'Substrate':[substrate]}); # control is not subtracted
    
    
    classical = GrowthMetrics(growth)
    classical.Classical(gompertz)
    classical.inferClassicalDynamics()

    to_header = ['classical_r','classical_K','classical_AUC','classical_td'];
    to_index = classical.key.index[0]
    growth_summary.loc[to_index,to_header] = classical.key.loc[to_index,to_header].values

    gpr = GrowthMetrics(growth)
    gpr.GP()
    gpr.inferGPDynamics()

    to_header = ['GP_r','GP_K','GP_AUC','GP_td'];
    to_index = gpr.key.index[0]
    growth_summary.loc[to_index,to_header] = gpr.key.loc[to_index,to_header].values

In [ ]:
summary_df = summary.join(growth_summary).sort_values(['GP_td'],ascending=True)
summary_df.head(10)

In [ ]:
fig,ax = plt.subplots(figsize=[5,7]);

# grab all growth curves with at least 1.5 fold change (relative to negative control)
subset = list(summary_df[summary_df['Growth Fold']>1.5].Substrate.values)+list(['Negative Control']);

labels = summary_df[summary_df.isin({'Substrate':subset}).any(1)].Substrate
td = summary_df[summary_df.isin({'Substrate':subset}).any(1)].GP_td;

ax.barh(y=range(len(labels)),width=td,height=0.8,color=(0,0,0,0.65));

[ii.set(fontsize=20) for ii in ax.get_xticklabels()+ax.get_yticklabels()];

ax.set_xlabel('Doubling Time (Estimated by GP)',fontsize=20);
plt.setp(ax,yticks=range(len(labels)),yticklabels=labels);
ax.yaxis.grid(False)